In [1]:
import pandas as pd
import seaborn as sb
import numpy as np

import category_encoders as ce

from sklearn.preprocessing import LabelEncoder

from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder

from tabulate import tabulate

from sklearn.linear_model import LogisticRegression

In [2]:
#df = pd.read_csv('datasets/loan_test.csv', sep=';')
#df = df[df.columns.difference(['status'])]

#df_train = pd.read_csv('datasets/loan_train.csv', sep=';')
#train_status = df_train['status']
#train_data = df_train[df_train.columns.difference(['status'])]

In [3]:
#df.head()

In [4]:
#logreg = LogisticRegression(solver='liblinear', max_iter=1000, class_weight='balanced', C=0.25)
#logreg.fit(train_data, train_status)

#predictions = logreg.predict(df)

#print(predictions)

In [5]:
#df_final = df['account_id']
#df_final = pd.DataFrame({"Id" : df['loan_id'], "Predicted" : predictions})

#df_final.to_csv('test.csv', index=False)
#print(type(predictions))

# Data Preprocessing

## Cleaning data

### Accounts details data

In [6]:
df_accounts = pd.read_csv('datasets/account.csv', sep=';')

print('Frequency unique values:', df_accounts['frequency'].unique())

Frequency unique values: ['monthly issuance' 'issuance after transaction' 'weekly issuance']


In [7]:
# Using binary encoding on frequency column 

binary_encoder = ce.BinaryEncoder(cols=['frequency'],return_df=True)
df_accounts = binary_encoder.fit_transform(df_accounts)

In [8]:
# Formatting date column

df_accounts['date'] = '19' + df_accounts['date'].astype('str')
df_accounts['date'] = pd.to_datetime(df_accounts.date, format="%Y%m%d")
df_accounts = df_accounts.rename(columns={"date": "account_date"})

df_accounts.to_csv('clean_datasets/account.csv', index=False)
df_accounts.head()

,account_id,district_id,frequency_0,frequency_1,account_date
0,576,55,0,1,1993-01-01
1,3818,74,0,1,1993-01-01
2,704,55,0,1,1993-01-01
3,2378,16,0,1,1993-01-01
4,2632,24,0,1,1993-01-02


### Client details

**birth_number** 
- YYMMDD format for Men
- YYMM+50DD format for Women

In [9]:
df_clients = pd.read_csv('datasets/client.csv', sep=';')
df_clients.head()

,client_id,birth_number,district_id
0,1,706213,18
1,2,450204,1
2,3,406009,1
3,4,561201,5
4,5,605703,5


#### Create column for client's sex and reformat birth date 

**sex**
- 0 - Woman
- 1 - Man

In [10]:
df_clients['sex'] = df_clients['birth_number'].apply(lambda bn: "WOMAN" if ((bn/100)%100) >= 51 else "MAN")

df_clients['birth_date'] = df_clients['birth_number'].apply(lambda bn: pd.to_datetime("19" + str(bn - 5000), format="%Y%m%d") if ((bn/100)%100) >= 51 else pd.to_datetime("19" + str(bn)))

df_clients = df_clients.drop(columns=['birth_number'])
df_clients.to_csv('clean_datasets/client.csv', index=False)
df_clients.head()

,client_id,district_id,sex,birth_date
0,1,18,WOMAN,1970-12-13
1,2,1,MAN,1945-02-04
2,3,1,WOMAN,1940-10-09
3,4,5,MAN,1956-12-01
4,5,5,WOMAN,1960-07-03


### Disposition data

In [11]:
df_disp = pd.read_csv('datasets/disp.csv', sep=';')
df_disp.head()

,disp_id,client_id,account_id,type
0,1,1,1,OWNER
1,2,2,2,OWNER
2,3,3,2,DISPONENT
3,4,4,3,OWNER
4,5,5,3,DISPONENT


In [12]:
print('All unique Disposition types: ', df_disp['type'].unique())

All unique Disposition types:  ['OWNER' 'DISPONENT']


#### Label encoding 'type' column

- 0 - OWNER
- 1 - DISPONENT

In [13]:
#df_disp['type'] = df_disp['type'].apply(lambda x: 0 if x == "OWNER" else 1)
#df_disp.head()

df_disp.to_csv('clean_datasets/disp.csv', index=False)

### District data

In [14]:
df_district = pd.read_csv('datasets/district.csv', sep=';')
df_district.head()

,code,name,region,no. of inhabitants,no. of municipalities with inhabitants < 499,no. of municipalities with inhabitants 500-1999,no. of municipalities with inhabitants 2000-9999,no. of municipalities with inhabitants >10000,no. of cities,ratio of urban inhabitants,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,2616,3040


In [15]:
print(len(df_district['region'].unique()), ' different regions: ', df_district['region'].unique())

8  different regions:  ['Prague' 'central Bohemia' 'south Bohemia' 'west Bohemia' 'north Bohemia'
 'east Bohemia' 'south Moravia' 'north Moravia']


In [16]:
df_district['region'] = df_district['region'].astype('category')

print(df_district.columns)

Index(['code ', 'name ', 'region', 'no. of inhabitants',
       'no. of municipalities with inhabitants < 499 ',
       'no. of municipalities with inhabitants 500-1999',
       'no. of municipalities with inhabitants 2000-9999 ',
       'no. of municipalities with inhabitants >10000 ', 'no. of cities ',
       'ratio of urban inhabitants ', 'average salary ',
       'unemploymant rate '95 ', 'unemploymant rate '96 ',
       'no. of enterpreneurs per 1000 inhabitants ',
       'no. of commited crimes '95 ', 'no. of commited crimes '96 '],
      dtype='object')


Some column names have a space at the end of the string. And 'region' column should be encoded.

In [17]:
# Using binary encoding on region column 

binary_encoder = ce.BinaryEncoder(cols=['region'],return_df=True)
df_district = binary_encoder.fit_transform(df_district)

In [18]:
for column in df_district.columns:
    df_district = df_district.rename(columns={column : column.strip()})


df_district = df_district.drop(columns=['name'])
df_district.head()

,code,region_0,region_1,region_2,region_3,no. of inhabitants,no. of municipalities with inhabitants < 499,no. of municipalities with inhabitants 500-1999,no. of municipalities with inhabitants 2000-9999,no. of municipalities with inhabitants >10000,no. of cities,ratio of urban inhabitants,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96
0,1,0,0,0,1,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1,2,0,0,1,0,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159,2674
2,3,0,0,1,0,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824,2813
3,4,0,0,1,0,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244,5892
4,5,0,0,1,0,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,2616,3040


#### Looking for Missing Values

In [19]:
df_district.isnull().values.any()

missing = False

for column in df_district.drop(columns=['region_0', 'region_1', 'region_2', 'region_3']).columns:
    if df_district[column].isnull().values.any():
        missing = True
        break
        
if not missing:
    print("No missing values in the data frame")
else:
    print("Missing values present in data frame")

No missing values in the data frame


Now that we have verified that there are no missing values in the district dataset, we must check that every column except `region` have **numeric** values

In [20]:
columns = []
is_numeric = []

for column in df_district.drop(columns=['region_0', 'region_1', 'region_2', 'region_3']).columns:
    columns.append(column)
    is_numeric.append(np.issubdtype(df_district[column].dtype, np.number))


df = pd.DataFrame({'Column' : columns,
                   'Is All Numeric' : is_numeric})
print(tabulate(df, headers='keys', tablefmt='psql'))

+----+--------------------------------------------------+------------------+
|    | Column                                           | Is All Numeric   |
|----+--------------------------------------------------+------------------|
|  0 | code                                             | True             |
|  1 | no. of inhabitants                               | True             |
|  2 | no. of municipalities with inhabitants < 499     | True             |
|  3 | no. of municipalities with inhabitants 500-1999  | True             |
|  4 | no. of municipalities with inhabitants 2000-9999 | True             |
|  5 | no. of municipalities with inhabitants >10000    | True             |
|  6 | no. of cities                                    | True             |
|  7 | ratio of urban inhabitants                       | True             |
|  8 | average salary                                   | True             |
|  9 | unemploymant rate '95                            | False            |

It seems that two columns that should be numeric have some not numeric values. It could be that some values are badly formatted numbers or invalid values that make this entry count as a missing value.

In [21]:
df_not_numeric = pd.DataFrame({"unemploymant rate '95":df_district["unemploymant rate '95"].values, "no. of commited crimes '95": df_district["no. of commited crimes '95"].values})

df_not_numeric["not_numeric1"] = df_not_numeric["unemploymant rate '95"].apply(lambda x: x if not x.replace('.', '', 1).isdigit() else "")
df_not_numeric["not_numeric2"] = df_not_numeric["no. of commited crimes '95"].apply(lambda x: x if not x.replace('.', '', 1).isdigit() else "")

print("Not numeric values in 'unemploymant rate '95': ", set(df_not_numeric["not_numeric1"].unique()) - set(['']))
print("Not numeric values in 'no. of commited crimes '95': ",set(df_not_numeric["not_numeric2"].unique()) - set(['']))

Not numeric values in 'unemploymant rate '95':  {'?'}
Not numeric values in 'no. of commited crimes '95':  {'?'}


As there are values in these columns marked as '?', we assume them as missing values and should remove them, for a correct missing values handling later on.

In [22]:
df_district["unemploymant rate '95"] = df_district["unemploymant rate '95"].apply(lambda x: x if not x == '?' else None)
df_district["no. of commited crimes '95"] = df_district["no. of commited crimes '95"].apply(lambda x: x if not x == '?' else None)

print("Missing values in 'unemploymant rate '95':", df_district["unemploymant rate '95"].isnull().values.sum())
print("Missing values in 'no. of commited crimes '95': ", df_district["no. of commited crimes '95"].isnull().values.sum())

Missing values in 'unemploymant rate '95': 1
Missing values in 'no. of commited crimes '95':  1


In [23]:
# Sum crimes 95 and 96

#df_district["no. of commited crimes '95"] = pd.to_numeric(df_district["no. of commited crimes '95"])
#df_district["no. of commited crimes '96"] = pd.to_numeric(df_district["no. of commited crimes '96"])

#df_district["no. of commited crimes '95"] += df_district["no. of commited crimes '96"] 
#df_district = df_district.drop(columns=["no. of commited crimes '96"]).rename(columns={"no. of commited crimes '95": "no. of commited crimes '95-96"})

In [24]:
imputer = KNNImputer(n_neighbors=5, weights="uniform")

df_district = pd.DataFrame(imputer.fit_transform(df_district), columns = df_district.columns)

for column in (set(df_district.columns) - set(("ratio of urban inhabitants", "unemploymant rate '95", "unemploymant rate '96)"))):
    df_district[column] = df_district[column].astype('int')

In [25]:
df_district = df_district.rename(columns={"code":"district_id"})
df_district.to_csv('clean_datasets/district.csv', index=False)

### Loan Data

In [26]:
df_loan_train = pd.read_csv('datasets/loan_train.csv', sep=";")
df_loan_test = pd.read_csv('datasets/loan_test.csv', sep=";")

In [27]:
df_loans = pd.concat([df_loan_train.drop(columns=["status"]), df_loan_test.drop(columns=["status"])])

df_loans.head()

,loan_id,account_id,date,amount,duration,payments
0,5314,1787,930705,96396,12,8033
1,5316,1801,930711,165960,36,4610
2,6863,9188,930728,127080,60,2118
3,5325,1843,930803,105804,36,2939
4,7240,11013,930906,274740,60,4579


#### Loan dates from number to datetime format

In [28]:
df_loans.date = df_loans.date.apply(lambda x: pd.to_datetime("19" + str(x), format="%Y%m%d"))
df_loans = df_loans.rename(columns={"date": "loan_date"})
df_loans.loan_date = df_loans.loan_date.astype('datetime64[ns]')
df_loans.head()

,loan_id,account_id,loan_date,amount,duration,payments
0,5314,1787,1993-07-05,96396,12,8033
1,5316,1801,1993-07-11,165960,36,4610
2,6863,9188,1993-07-28,127080,60,2118
3,5325,1843,1993-08-03,105804,36,2939
4,7240,11013,1993-09-06,274740,60,4579


In [29]:
df_loans_train_clean = df_loans[df_loans['loan_id'].isin(df_loan_train['loan_id'].values)]
df_loans_train_clean['status'] = df_loan_train['status']
df_loans_train_clean.to_csv('clean_datasets/loans_train.csv', index=False)

df_loans_test_clean = df_loans[df_loans['loan_id'].isin(df_loan_test['loan_id'].values)]
df_loans_test_clean['status'] = df_loan_test['status']
df_loans_test_clean.to_csv('clean_datasets/loans_test.csv', index=False)

<ipython-input-29-ac172a8b49de>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loans_train_clean['status'] = df_loan_train['status']
<ipython-input-29-ac172a8b49de>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loans_test_clean['status'] = df_loan_test['status']


## Classification

In [30]:
df_loan_train = pd.read_csv('clean_datasets/loans_train.csv', parse_dates=['loan_date'], date_parser=pd.to_datetime)
df_loan_test = pd.read_csv('clean_datasets/loans_test.csv', parse_dates=['loan_date'], date_parser=pd.to_datetime)

In [31]:
df_accounts_test = pd.read_csv('clean_datasets/account.csv', parse_dates=['account_date'], date_parser=pd.to_datetime)

In [32]:
df_districts_test = pd.read_csv('clean_datasets/district.csv', dtype={"region":"category"})
df_districts_test = df_districts_test.reset_index()

In [33]:
merge_train = pd.merge(df_loan_train, df_accounts_test, on = 'account_id', how='left')
merge_train = pd.merge(merge_train, df_districts_test, on = 'district_id',how='left')

In [34]:
merge_test = pd.merge(df_loan_test, df_accounts_test, on = 'account_id', how='left')
merge_test = pd.merge(merge_test, df_districts_test, on = 'district_id',how='left')

In [35]:
merge_train.head()

,loan_id,account_id,loan_date,amount,duration,payments,status,district_id,frequency_0,frequency_1,...,no. of municipalities with inhabitants 2000-9999,no. of municipalities with inhabitants >10000,no. of cities,ratio of urban inhabitants,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96
0,5314,1787,1993-07-05,96396,12,8033,-1,30,1,1,...,8,2,10,81.8,9650,3.38,3,100,2985,2804
1,5316,1801,1993-07-11,165960,36,4610,1,46,0,1,...,7,3,10,73.5,8369,1.79,2,117,2854,2618
2,6863,9188,1993-07-28,127080,60,2118,1,45,0,1,...,6,1,5,53.5,8390,2.28,2,132,2080,2122
3,5325,1843,1993-08-03,105804,36,2939,1,12,0,1,...,6,1,6,58.0,8754,3.83,4,137,3804,3868
4,7240,11013,1993-09-06,274740,60,4579,1,1,1,1,...,0,1,1,100.0,12541,0.29,0,167,85677,99107


In [36]:
merge_train['time_since_create'] = (merge_train["loan_date"] - merge_train["account_date"]).dt.days
merge_train = merge_train.drop(columns=["account_date", "loan_date", "index"])

merge_test['time_since_create'] = (merge_test["loan_date"] - merge_test["account_date"]).dt.days
merge_test = merge_test.drop(columns=["account_date", "loan_date", "index"])

In [37]:
merge_train.head()

,loan_id,account_id,amount,duration,payments,status,district_id,frequency_0,frequency_1,region_0,...,no. of municipalities with inhabitants >10000,no. of cities,ratio of urban inhabitants,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96,time_since_create
0,5314,1787,96396,12,8033,-1,30,1,1,0,...,2,10,81.8,9650,3.38,3,100,2985,2804,105
1,5316,1801,165960,36,4610,1,46,0,1,0,...,3,10,73.5,8369,1.79,2,117,2854,2618,148
2,6863,9188,127080,60,2118,1,45,0,1,0,...,1,5,53.5,8390,2.28,2,132,2080,2122,170
3,5325,1843,105804,36,2939,1,12,0,1,0,...,1,6,58.0,8754,3.83,4,137,3804,3868,185
4,7240,11013,274740,60,4579,1,1,1,1,0,...,1,1,100.0,12541,0.29,0,167,85677,99107,204


In [38]:
merge_train.to_csv("merge_train.csv", index=False)
merge_test.to_csv("merge_test.csv", index=False)

In [40]:
from sklearn import svm

clf = svm.SVC(kernel='linear')

clf.fit(merge_train.drop(columns=["status", "loan_id", "account_id", "district_id"]), merge_train['status'])

SVC(kernel='linear')

In [42]:
y_pred = clf.predict(merge_test.drop(columns=['status', "loan_id", "account_id", "district_id"]))
print(y_pred)

[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1

In [44]:
logreg = LogisticRegression(solver='liblinear', max_iter=1000, class_weight='balanced', C=0.25)
logreg.fit(merge_train.drop(columns=["status", "loan_id", "account_id", "district_id"]), merge_train['status'])

predictions = logreg.predict(merge_test.drop(columns=['status', "loan_id", "account_id", "district_id"]))

print(predictions)

[ 1 -1  1  1  1  1  1 -1  1  1 -1  1 -1 -1 -1 -1 -1 -1 -1  1  1 -1  1  1
  1  1  1  1 -1 -1 -1  1  1  1  1  1  1 -1  1  1  1 -1 -1  1  1  1  1  1
  1  1  1 -1 -1 -1  1  1  1 -1  1  1  1 -1  1  1  1 -1 -1  1 -1  1  1 -1
 -1  1 -1 -1 -1  1 -1  1 -1 -1  1  1 -1  1  1  1 -1  1 -1 -1  1  1 -1 -1
  1 -1  1  1 -1 -1  1  1  1 -1  1  1 -1 -1  1  1  1 -1  1 -1  1 -1  1 -1
  1  1 -1  1  1 -1 -1  1  1  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1  1 -1
  1  1 -1 -1  1  1  1  1  1  1 -1  1  1  1  1 -1 -1  1 -1 -1  1  1 -1 -1
  1 -1  1  1 -1  1 -1  1  1  1  1  1  1 -1  1  1  1  1  1  1 -1  1 -1 -1
 -1  1 -1 -1 -1  1 -1 -1 -1  1 -1  1  1  1 -1  1  1 -1  1  1 -1  1  1  1
  1  1  1 -1  1 -1  1  1 -1 -1 -1 -1  1  1 -1  1  1  1  1  1 -1 -1  1  1
  1 -1  1  1  1  1  1  1  1  1  1 -1 -1  1  1  1 -1  1 -1  1 -1  1  1  1
  1  1  1  1  1 -1  1  1  1  1  1  1  1 -1 -1 -1 -1  1  1  1  1  1  1  1
 -1  1  1  1  1  1  1  1  1  1  1  1 -1 -1 -1  1 -1  1  1 -1 -1  1  1 -1
 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1  1  1  1 -1  1

In [45]:
df_final = pd.DataFrame({"Id" : merge_test['loan_id'], "Predicted" : predictions})

df_final.to_csv('test.csv', index=False)
print(type(predictions))

<class 'numpy.ndarray'>
